In [76]:
import rhkpy as rk
import xarray as xr
import hvplot.xarray
import numpy as np
import holoviews as hv
import panel as pn

In [63]:
# Create a sample xarray DataArray with random 2D data
data = np.random.rand(10, 10)
coords = {'y': np.arange(10), 'x': np.arange(10)}
dataarray = xr.DataArray(data, coords=coords, dims=['y', 'x'])

In [64]:
# List to store clicked coordinates
clicked_coords = []

# Function to handle taps
def tap_handler(x, y):
    global plot
    
    if len(clicked_coords) < 2:
        clicked_coords.append((x, y))
        
    if len(clicked_coords) == 2:
        annotations = [hv.Text(x, y, f"({x:.2f}, {y:.2f})").opts(text_color='white') for x, y in clicked_coords]
        overlay = plot * hv.Overlay(annotations)
        hv.extension('bokeh')
        display(overlay)
    else:
        display(plot)

# Function to use stored coordinates later
def process_coords():
    for x, y in clicked_coords:
        # Process coordinates here
        print(f"Processing: x={x}, y={y}")
        print(f"Data value: {dataarray.sel(x=x, y=y, method='nearest').item()}")

In [65]:
# Attach tap stream
plot = dataarray.hvplot.image(aspect = 'equal')
tap = hv.streams.Tap(transient = True, source = plot)
tap.add_subscriber(tap_handler)

In [66]:
plot

:Image   [x,y]   (value)

In [68]:
process_coords()

Processing: x=1.4866658528645833, y=0.8799999237060547
Data value: 0.7563586167798617
Processing: x=2.786665852864583, y=1.213333257039388
Data value: 0.33308843819392586


## Updating in realtime

In [77]:
# List to store clicked coordinates
clicked_coords = []

# Function to handle taps
def tap_handler(x, y):
    if x is not None and y is not None and len(clicked_coords) < 2:
        clicked_coords.append((x, y))
        print(f"Stored coordinates: {clicked_coords}")

# Function to return line based on clicked_coords
def draw_line():
    if len(clicked_coords) == 2:
        x1, y1 = clicked_coords[0]
        x2, y2 = clicked_coords[1]
        return hv.Path([[(x1, y1), (x2, y2)]], vdims=[]).opts(color='red', line_width=2)
    else:
        return hv.Path([], vdims=[])

# Function to reset clicked coordinates
def reset_coords(event):
    clicked_coords.clear()
    print("Coordinates reset.")

In [78]:
# Create the initial plot
plot = dataarray.hvplot.image(rasterize=True, aspect='equal')

# DynamicMap to update line
dynamic_line = hv.DynamicMap(draw_line)

# Create an overlay of the plot and line
overlay = plot * dynamic_line

# Attach tap stream
tap = hv.streams.Tap(transient=True, source=plot)
tap.add_subscriber(tap_handler)

# Create Reset button
reset_button = pn.widgets.Button(name="Reset", width=100)
reset_button.on_click(reset_coords)

# Layout
layout = pn.Row(pn.Column(reset_button), overlay)

In [79]:
layout

Coordinates reset. 
Coordinates reset.

Row
    [0] Column
        [0] Button(name='Reset', width=100)
    [1] HoloViews(DynamicMap, sizing_mode='fixed')

In [83]:
process_coords()

Processing: x=3.1466664632161456, y=5.266041564941406
Data value: 0.6550144212190322


In [86]:
process_coords()

Processing: x=6.179999796549479, y=8.752708435058594
Data value: 0.04451167057404248
